In [1]:
import pandas as pd

In [2]:
model_name = 'biological_appearance_hamming'
electrode_region = 'parietal'
constant = 'naive'
variable = 'video'

In [3]:
mult_reg_results, _, _  = pd.read_pickle('/auto/data2/oelmas/RSA_Results_V1//Naive/Video/correlation_hamming_regression_results.pkl')

In [4]:
from statsmodels.sandbox.stats.multicomp import multipletests

In [5]:
sig_idx = multipletests(mult_reg_results["p_value"], alpha=0.05, method='bonferroni')[0]
mult_reg_results['significant'] = sig_idx

In [6]:
er_idx = mult_reg_results.electrode_region==electrode_region
ex_idx = mult_reg_results.experiment_type==constant
st_idx = mult_reg_results.stimuli_type==variable

reg_df = mult_reg_results[ex_idx & st_idx & er_idx]

In [7]:
sig_reg_df = reg_df.loc[reg_df['significant']]

In [8]:
import plotly.express as px
import plotly.graph_objs as go
import plotly.offline as po

In [9]:
fig = px.scatter(sig_reg_df, 
                 x="time", y="model_name",  color="model_name",
                title="Timepoints where the regressors reached statistical significance (Bonferroni corr.)")

fig.update_yaxes(title_text="Regressors")
fig.update_xaxes(range=[-200, 600], title_text="Time ms (relative to stimulus onset)")


po.plot(fig, filename = "outputs/SCATTER.html", auto_open=False)

'outputs/SCATTER.html'

In [10]:
# Plot on the correlational plot
avg_sbj_results, eeg_dist, model_dist  = pd.read_pickle('/auto/data2/oelmas/RSA_Results_V1//Naive/Video/correlation_hamming_avg_subjects_results.pkl')

In [11]:
rdm_df = avg_sbj_results

In [12]:
df_to_plot = rdm_df[  (rdm_df["model_name"] == model_name) 
                    & (rdm_df["electrode_region"] == electrode_region)
                    & (rdm_df["experiment_type"] == constant) 
                    & (rdm_df["stimulus_type"] == variable)]

In [13]:
fig = go.Figure()
 
# Plots the kendall tau curve
fig.add_trace(go.Scatter(x=df_to_plot["time"],
                        y=df_to_plot["kendall_tau"],
                        mode='lines'))


fig.add_shape(
        # Rectangle reference to the plot
            type="rect",
            xref="x",
            yref="paper",
            x0=min(reg_df.loc[reg_df['significant']].time),
            y0=0,
            x1=max(reg_df.loc[reg_df['significant']].time),
            y1=1,
            opacity=0.4,
            layer="below",
            line_width=0,
            fillcolor="LightSalmon",
            name='Model reached significance in the multiple regression analysis',
        )

fig.update_yaxes(title_text="Kendall-tau")
fig.update_xaxes(range=[-200, 600], title_text="Time ms (relative to stimulus onset)")

fig.update_layout(
    height=800,
    title_text='Correlation of the ' + model_name + ' model',
    showlegend=True
)

po.plot(fig, filename = "outputs/TEST.html", auto_open=False)

'outputs/TEST.html'

In [14]:
#############################################################
#############################################################
# variance inflation factor
VIF =  pd.read_csv('/auto/data2/oelmas/RSA_Results_V1//Naive/Video/VIF_Results.csv', index_col=[0])
print(VIF)

   VIF Factor                         model_name
0    1.062114            action_category_hamming
1    2.062207                      agent_hamming
2    2.557533    flowModel_10_frames_correlation
3    3.600027  gabor_video_10_frames_correlation
